In [2]:
import pandas as pd
import itertools
import numpy as np
from scipy.stats import chi2_contingency
import prince

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

file_path = 'C:/Users/Venner/Desktop/spreadsheet.xlsx'

opportunities_df = pd.read_excel(file_path, sheet_name='opportunites')
orders_df = pd.read_excel(file_path, sheet_name='orders')

merged_df = pd.merge(opportunities_df, orders_df, left_on='SR_ID', right_on='SR_ID', how='inner')

columns_to_encode = ['SOURCE1', 'STATUS1', 'GEO_REGION_ID', 'SOURCE_DETAIL', 'DISPOSITION',
                     'SHIFTING_TYPE', 'DROP_GEO_REGION_ID', 'PLATFORM', 'STATUS']

data_encoded_full_df = pd.get_dummies(merged_df[columns_to_encode], drop_first=True)

data_encoded_clean_df = data_encoded_full_df.dropna()

X_full_clean = data_encoded_clean_df.drop(columns=[col for col in data_encoded_clean_df.columns if 'STATUS_' in col])  
y_full_clean = data_encoded_clean_df[[col for col in data_encoded_clean_df.columns if 'STATUS_' in col]]  

In [4]:
def cramers_v(x, y):
#     calculate Cramér's V between categories
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r - 1, k - 1))))

In [5]:
data = data_encoded_clean_df  # Using the cleaned dataset from coefficient_code2
data.head()

,SOURCE1,STATUS1,GEO_REGION_ID,DROP_GEO_REGION_ID,SOURCE_DETAIL_Inbound call,SOURCE_DETAIL_Mobile Website,DISPOSITION_Follow up required,DISPOSITION_Language Barrier,DISPOSITION_Not Interested,DISPOSITION_Quotation Shared,...,DISPOSITION_Requirement matches,SHIFTING_TYPE_intra_city,PLATFORM_ios,STATUS_completed,STATUS_open,STATUS_shifting_started,STATUS_supervisor_accepted,STATUS_supervisor_assigned,STATUS_trip_started,STATUS_vendor_accepted
0,2,4,8.0,8.0,False,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False
1,0,4,6.0,6.0,False,True,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
3,2,4,8.0,8.0,False,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False
4,0,4,3.0,3.0,False,True,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False
6,0,3,3.0,3.0,False,True,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False


In [6]:
data.columns

Index(['SOURCE1', 'STATUS1', 'GEO_REGION_ID', 'DROP_GEO_REGION_ID',
       'SOURCE_DETAIL_Inbound call', 'SOURCE_DETAIL_Mobile Website',
       'DISPOSITION_Follow up required', 'DISPOSITION_Language Barrier',
       'DISPOSITION_Not Interested', 'DISPOSITION_Quotation Shared',
       'DISPOSITION_RNR', 'DISPOSITION_Requirement matches',
       'SHIFTING_TYPE_intra_city', 'PLATFORM_ios', 'STATUS_completed',
       'STATUS_open', 'STATUS_shifting_started', 'STATUS_supervisor_accepted',
       'STATUS_supervisor_assigned', 'STATUS_trip_started',
       'STATUS_vendor_accepted'],
      dtype='object')

In [7]:
cols = ['SOURCE1', 'STATUS1', 'GEO_REGION_ID', 'SOURCE_DETAIL', 'DISPOSITION',
                     'SHIFTING_TYPE', 'DROP_GEO_REGION_ID', 'PLATFORM', 'STATUS']

In [8]:
metrix_df = pd.DataFrame(columns=['col1','col2','corr'])
for col1, col2 in list(itertools.combinations(cols,2)):
    if col1 in data.columns and col2 in data.columns:
      metrix_df.loc[len(metrix_df)] = [col1, col2, cramers_v(data[col1], data[col2])]
metrix_df.sort_values(by='corr',ascending=False,inplace=True,ignore_index=True)
metrix_df

,col1,col2,corr
0,GEO_REGION_ID,DROP_GEO_REGION_ID,0.947194
1,SOURCE1,DROP_GEO_REGION_ID,0.054723
2,SOURCE1,GEO_REGION_ID,0.054625
3,STATUS1,DROP_GEO_REGION_ID,0.049562
4,STATUS1,GEO_REGION_ID,0.047212
5,SOURCE1,STATUS1,0.041549


GEO_REGION_ID, DROP_GEO_REGION_ID, SHIFTING_TYPE_score have high correlations

# MCA

In [15]:

X = pd.DataFrame({})
list_col = ['SOURCE1', 'STATUS1', 'GEO_REGION_ID', 'SOURCE_DETAIL', 'DISPOSITION', 'SHIFTING_TYPE', 'DROP_GEO_REGION_ID', 'PLATFORM', 'STATUS']
for col in list_col:
    X[col] = merged_df[col]
y = merged_df['STATUS'] 


In [16]:
mca = prince.MCA(n_components=30)
X_mca = mca.fit_transform(X)
X_mca

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-0.136732,0.449251,-0.291582,-0.078684,-0.138206,0.322440,-0.312002,0.121099,0.130312,1.435809,...,-0.050851,-0.102661,-0.035586,-0.129053,0.037852,0.041448,-0.094803,0.086068,-0.025611,-0.106016
1,-0.071341,-0.007508,-0.027089,-0.137117,-0.031139,-0.002058,-0.065762,0.198911,0.348848,0.392677,...,-0.554783,-1.044303,0.355007,0.806567,0.314776,0.170773,-0.196103,-0.112613,0.328950,0.461296
2,0.022372,-0.481848,0.160154,-0.016982,-0.042845,-0.138261,0.054646,-0.001431,-0.101247,-0.121891,...,0.526409,-0.287788,0.401642,0.052666,0.047131,0.261437,0.049922,-0.554907,0.176306,0.072606
3,-0.136732,0.449251,-0.291582,-0.078684,-0.138206,0.322440,-0.312002,0.121099,0.130312,1.435809,...,-0.050851,-0.102661,-0.035586,-0.129053,0.037852,0.041448,-0.094803,0.086068,-0.025611,-0.106016
4,-0.089159,-0.232874,0.061508,-0.196994,-0.447582,0.140784,-0.072567,-0.268150,-0.287507,-0.499786,...,0.158287,-0.208159,-0.065170,0.052078,-0.034766,0.074918,0.104078,-0.131580,0.032369,0.044628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13342,-0.118939,0.373277,-0.250556,-0.072788,-0.126453,0.285512,-0.286991,0.102422,0.139318,1.243761,...,-0.057251,-0.279693,0.119609,-0.293769,-0.083636,-0.145242,0.110254,0.030640,-0.176120,0.589261
13343,-0.119492,0.395834,-0.114495,-0.276026,-0.341768,0.243458,-0.112696,-0.151495,-0.042479,-0.425943,...,0.206338,-0.061445,0.123826,-0.148102,-0.041218,-0.323777,0.285093,-0.125755,-0.191866,0.752384
13344,-0.147047,0.152475,-0.064621,-0.027563,-0.422269,0.185972,-0.180964,-0.315440,-0.348380,0.219837,...,-0.806148,-1.881396,0.488922,1.345909,-0.074300,0.182123,-0.886926,1.035130,-0.519908,0.005013
13345,0.080761,-0.456169,0.183079,-0.004872,0.338286,-0.388644,0.218685,0.118611,0.255180,-0.162258,...,0.225381,0.101568,-0.129170,0.021733,0.006614,-0.492148,0.030006,-0.077218,0.007165,0.066126


In [17]:
# the max correlation after mca
corrs = X_mca.corr().values
sorted(set([corr for corrs_list in corrs for corr in corrs_list]))[-2]

np.float64(0.040823625348132525)

# Build Model

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, f1_score, accuracy_score, precision_score, recall_score

In [32]:
X = pd.DataFrame({})
list_col = ['SOURCE1', 'STATUS1', 'GEO_REGION_ID', 'SOURCE_DETAIL', 'DISPOSITION', 'SHIFTING_TYPE', 'DROP_GEO_REGION_ID', 'PLATFORM', 'STATUS']
for col in list_col:
    X[col] = merged_df[col]
y = merged_df['STATUS'] 


## Linear regression

In [33]:
# Convert categorical columns in X to numeric using one-hot encoding
X_encoded = pd.get_dummies(X)

# Handle missing values by imputing with the mean
imputer = SimpleImputer(strategy='mean')
X_encoded_imputed = imputer.fit_transform(X_encoded)

# Convert y (the target) from strings to numeric labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_encoded_imputed, y_encoded, random_state=40, test_size=0.2)

# Train the model
L1 = LinearRegression()
L1.fit(X_train, y_train)

# Predict and calculate R^2 score
y_pred_L1 = L1.predict(X_test)
r2_L1 = r2_score(y_test, y_pred_L1)
r2_L1

1.0

The result is very bad because there is no linear relation between features and target.

## Decision Tree

In [34]:
def eva(y_pred,y_test):
    mse = mean_squared_error(y_pred,y_test)
    # r2 = r2_score(y_pred,y_test)
    f1 = f1_score(y_pred,y_test, average='macro')
    acc = accuracy_score(y_pred,y_test)
    rec = recall_score(y_pred,y_test, average='macro')
    pre = precision_score(y_pred,y_test, average='macro')
    return [mse,f1,acc,rec,pre]

metric_df = pd.DataFrame(data=[],index=['mse','f1 score','accuracy','recall','precision'])

In [35]:
DT1 = DecisionTreeClassifier()
DT1.fit(X_train, y_train)
y_pred_DT1 = DT1.predict(X_test)

metric_df['Decision Tree'] = eva(y_pred_DT1, y_test)


## Random Forest

In [36]:
RF1 = RandomForestClassifier()
RF1.fit(X_train, y_train)
y_pred_RF1 = RF1.predict(X_test)

metric_df['Random Forest'] = eva(y_pred_RF1, y_test)

## SVM

In [37]:
svm1 = SVC()
svm1.fit(X_train, y_train)
y_pred_svm1 = svm1.predict(X_test)

metric_df['SVM'] = eva(y_pred_svm1, y_test)

d:\appppps\python\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## KNN

In [38]:
knn1 = KNeighborsClassifier()
knn1.fit(X_train, y_train)
y_pred_knn1 = knn1.predict(X_test)

metric_df['KNN'] = eva(y_pred_knn1, y_test)

In [39]:
metric_df

,Decision Tree,Random Forest,SVM,KNN
mse,0.0,0.0,0.048689,0.492135
f1 score,1.0,1.0,0.873190,0.868492
accuracy,1.0,1.0,0.997753,0.966667
recall,1.0,1.0,0.874104,0.981908
precision,1.0,1.0,0.872292,0.808491


In [40]:
print(metric_df)

           Decision Tree  Random Forest       SVM       KNN
mse                  0.0            0.0  0.048689  0.492135
f1 score             1.0            1.0  0.873190  0.868492
accuracy             1.0            1.0  0.997753  0.966667
recall               1.0            1.0  0.874104  0.981908
precision            1.0            1.0  0.872292  0.808491
